In [9]:
import os
import json
import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from torch.utils.data import DataLoader
from PIL import Image


In [15]:
# Need to load the images and annotations.

# Create a dataset class for COCO128

class Coco128Dataset(torch.utils.data.Dataset):
    def __init__(self, root, labels_dir, transforms=None):
        self.root = root
        self.labels_dir = labels_dir
        self.transforms = transforms
        self.imgs = sorted(os.listdir(root))
        self.labels = sorted(os.listdir(labels_dir))

    def __getitem__(self, index):
        img_path = os.path.join(self.root, self.imgs[index])
        label_path = os.path.join(self.labels_dir, self.labels[index])
        
        img = Image.open(img_path).convert("RGB")
        
        # Read label file
        boxes = []
        labels = []
        with open(label_path) as f:
            for line in f.readlines():
                x = line.strip().split()
                label, x_center, y_center, width, height = map(float, x)
                xmin = (x_center - width / 2)
                ymin = (y_center - height / 2)
                xmax = (x_center + width / 2)
                ymax = (y_center + height / 2)
                
                boxes.append([xmin, ymin, xmax, ymax])
                labels.append(int(label))
        
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)



In [18]:
def get_transform(train):
    transforms = []
    transforms.append(F.to_tensor)
    return torchvision.transforms.Compose(transforms)

# load dataset
dataset = Coco128Dataset(root="coco128/images", labels_dir="coco128/labels", transforms=get_transform(train=True))

data_loader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=4)


In [19]:
#Initilize the model
model = fasterrcnn_resnet50_fpn(pretrained=True)


In [21]:
#Train Model & parameters
#forwards pass, loss, backwards pass, update parameters

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# move the model to the right device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Epochs
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    i = 0
    epoch_loss = 0
    for images, targets in data_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # forward pass
        loss_dict = model(images, targets)
        
        # total loss
        losses = sum(loss for loss in loss_dict.values())
        
        # backward pass
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        # update epoch loss
        epoch_loss += losses.item()
        i += 1

        # print interim results
        if i % 10 == 0:
            print(f"Iteration #{i} loss: {losses.item()}")

    # update the learning rate
    lr_scheduler.step()

    print(f"Epoch #{epoch+1} Loss: {epoch_loss/i}")

print("Training complete!")


Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "<string>", line 1, in <module>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "<string>", line 1, in <module>
  File "/Users/pazzo/opt/miniconda3/envs/mlenv/lib/python3.8/multiprocessing/spawn.py", line 116, in spawn_main
  File "/Users/pazzo/opt/miniconda3/envs/mlenv/lib/python3.8/multiprocessing/spawn.py", line 116, in spawn_main
  File "/Users/pazzo/opt/miniconda3/envs/mlenv/lib/python3.8/multiprocessing/spawn.py", line 116, in spawn_main
  File "/Users/pazzo/opt/miniconda3/envs/mlenv/lib/python3.8/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
    exitcode = _main(fd, parent_sentinel)
  File "/Users/pazzo/opt/miniconda3/envs/mlenv/lib/python3.8/multiprocessing/spawn.py", line 126, in _main
    exitcode = _main(fd, parent_sentinel)  File "/Users/pazzo/opt

RuntimeError: DataLoader worker (pid(s) 13971) exited unexpectedly